In [41]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.prompts import PromptTemplate
import json
import requests

api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERP_API_KEY")
# os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# os.environ["WANDB_PROJECT"] = "llmapps"

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate

In [10]:
from langchain.document_loaders import PyPDFLoader


loader = PyPDFLoader("../new_pdf/Software Developer.pdf")
final = loader.load()
resume = final[0].page_content
resume

'ROBIN GRAHAM\nSOFTWARE DEVELOPERrobin.graham@gmail.com\n202-555-0101\nPasadena, CA, US\nlinkedin.com/in/robin\nSUMMARY Working in tech companies since 2013, I succeed to grow my programming skills in\ndifferent areas. I participate in projects which developing applications for Windows,\nAndroid, Web, and firmware for hardware devices also.\nMy practical experience is with C and C++ languages and I am motivated to mastering\nthose skills and to enrich my knowledge with a new one.\nSKILLS AND\nTECHNOLOGIESC / C++\nHMTL, CSS, Javascript, PHP, MySQL\nJava, XML\nSVN, Mercurial\nIAR, Eclipse, Embarcadero RAD Studio XE5, Android Studio\nJSON, XML\nEXPERIENCE\nOpenXcell, NY Sep 2016 -PresentEmbedded Software Engineer\nDevelop and support device firmware\nImplementing new features\nRe-writing a radio protocol using for communication between devices\nImplement web sockets communication encrypted by SSL\nSyberry Corporation, CA Oct 2015 -Sep 2016Software Engineer\nWrite a drive for a RTC module\

In [13]:
from typing import Optional, List
from pydantic import BaseModel, Field

class Applicant(BaseModel):
    Name: str
    Contact: str
    Location: str
    University: str
    Degree: str
    Major: str
    Graduation_Year: int
    Skills: list[str]
    Certifications: Optional[list[str]]
    Languages: Optional[list[str]]
    Role: str
    Company: str
    Start_Date: str
    End_Date: str
    Experience: int
    Responsibilities: list[str]
    Achievements: Optional[list[str]]
    Projects: Optional[list[str]]
    Hobbies: Optional[list[str]]

In [63]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [64]:
llm = ChatOpenAI(temperature=0.0)

In [65]:
# Set up a parser + inject instructions into the prompt template.
pydantic_parser = PydanticOutputParser(pydantic_object=Applicant)

format_instructions = pydantic_parser.get_format_instructions()

In [66]:
template_string = """ You are a professional HR that and inforomation extractor that can analye the resume of the person and extract the contents as given in the instructions below

Resume: ```{resume}```

{format_instructions}
"""

In [67]:
prompt = ChatPromptTemplate.from_template(template=template_string)

messages = prompt.format_messages(resume =resume, 
                                format_instructions=format_instructions)

In [74]:
output = llm(messages)

In [84]:
Markdown(output.content)

{
  "Name": "ROBIN GRAHAM",
  "Contact": "robin.graham@gmail.com",
  "Location": "Pasadena, CA, US",
  "University": "Johns Hopkins University",
  "Degree": "Master of Computer Systems and Technologies",
  "Major": "Computer Systems and Technologies",
  "Graduation_Year": 2015,
  "Skills": [
    "C / C++",
    "HMTL, CSS, Javascript, PHP, MySQL",
    "Java, XML",
    "SVN, Mercurial",
    "IAR, Eclipse, Embarcadero RAD Studio XE5, Android Studio",
    "JSON, XML"
  ],
  "Certifications": [],
  "Languages": [
    "English",
    "Bulgarian",
    "French"
  ],
  "Role": "Embedded Software Engineer",
  "Company": "OpenXcell",
  "Start_Date": "Sep 2016",
  "End_Date": "Present",
  "Experience": 4,
  "Responsibilities": [
    "Develop and support device firmware",
    "Implementing new features",
    "Re-writing a radio protocol using for communication between devices",
    "Implement web sockets communication encrypted by SSL"
  ],
  "Achievements": [],
  "Projects": [],
  "Hobbies": []
}

In [83]:
the_brand = pydantic_parser.parse(output.content)



'ROBIN GRAHAM'

In [85]:
type(output.content)

str